In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import seaborn as sns

nltk.download('stopwords')


# Load your data
df = pd.read_csv('/content/Concat_Translated_Review.csv', encoding='utf-8')

df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Rating,Translated_Review
0,2,الشحن المثالي السريع جيد الجودة المعتاد حقا
1,2,مساعدة عملية شرا شفرة الخصم غير قادرة على تطبي...
2,2,روعة داءما مشكلة طلبية الجمنازيوم
3,2,توصيل سريع عالي الجودة على الموقع الءلكتروني ا...
4,2,جودة شرا الملابس الرياضية السهلة تم تسليمها بسرعة


In [2]:
df.isna().sum()

Rating                  0
Translated_Review    1558
dtype: int64

In [3]:
df = df.dropna()
df.isna().sum()

Rating               0
Translated_Review    0
dtype: int64

In [4]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('arabic'))
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

def preprocess(text):
    text = remove_stopwords(text)
    return text

# Preprocess the text
df['Translated_Review'] = df['Translated_Review'].apply(preprocess)

# Split into training and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(df['Translated_Review'], df['Rating'], test_size=0.2, random_state=42)

In [5]:
df

,Rating,Translated_Review
0,2,الشحن المثالي السريع جيد الجودة المعتاد
1,2,مساعدة عملية شرا شفرة الخصم قادرة تطبيق
2,2,روعة داءما مشكلة طلبية الجمنازيوم
3,2,توصيل سريع عالي الجودة الموقع الءلكتروني السهل...
4,2,جودة شرا الملابس الرياضية السهلة تم تسليمها بسرعة
...,...,...
79564,1,كتاب جيد وءن مملا الشي منتصف الكتاب
79565,1,ءول تجربة الخيال العلميالكثير المعلومات بقالب ...
79566,1,مرضي الافطار لذيذ يوجد قاءمة طعام الغرفةلم يتم...
79567,1,الرساءل واءل شوق كانت ءجمل مافي الرواية ءنها ك...


In [6]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers

# Train tokenizer on your data
tokenizer = Tokenizer(models.BPE())
trainer = trainers.BpeTrainer(vocab_size=32000, min_frequency=2)
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# Save the texts to a temporary file for training tokenizer
with open('texts.txt', 'w', encoding='utf-8') as f:
    for text in df['Translated_Review']:
        f.write(text + '\n')

tokenizer.train(['texts.txt'], trainer)
tokenizer.save("tokenizer.json")

# Load the tokenizer
tokenizer = Tokenizer.from_file("tokenizer.json")

def tokenize_texts(texts):
    encodings = tokenizer.encode_batch(texts)
    return {'input_ids': [encoding.ids for encoding in encodings]}

train_encodings = tokenize_texts(train_texts)
test_encodings = tokenize_texts(test_texts)


In [7]:
import torch
from torch.utils.data import DataLoader, Dataset

class ArabicSentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['Rating'] = torch.tensor(self.labels.iloc[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ArabicSentimentDataset(train_encodings, train_labels)
test_dataset = ArabicSentimentDataset(test_encodings, test_labels)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [8]:
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        self.encoding.requires_grad = False

        pos = torch.arange(0, max_len).unsqueeze(1).float()
        _2i = torch.arange(0, d_model, step=2).float()
        self.encoding[:, 0::2] = torch.sin(pos / (10000 ** (_2i / d_model)))
        self.encoding[:, 1::2] = torch.cos(pos / (10000 ** (_2i / d_model)))

    def forward(self, x):
        batch_size, seq_len, d_model = x.size()
        return x + self.encoding[:seq_len, :].to(x.device)


In [9]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, nhead, dropout=0.1):
        super(MultiHeadAttention, self).__init__()
        assert d_model % nhead == 0
        self.d_k = d_model // nhead
        self.nhead = nhead
        self.linear_q = nn.Linear(d_model, d_model)
        self.linear_k = nn.Linear(d_model, d_model)
        self.linear_v = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.out_proj = nn.Linear(d_model, d_model)

    def forward(self, query, key, value):
        batch_size = query.size(0)
        query = self.linear_q(query).view(batch_size, -1, self.nhead, self.d_k).transpose(1, 2)
        key = self.linear_k(key).view(batch_size, -1, self.nhead, self.d_k).transpose(1, 2)
        value = self.linear_v(value).view(batch_size, -1, self.nhead, self.d_k).transpose(1, 2)

        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(self.d_k)
        attn_weights = nn.Softmax(dim=-1)(scores)
        attn_weights = self.dropout(attn_weights)

        context = torch.matmul(attn_weights, value).transpose(1, 2).contiguous().view(batch_size, -1, self.nhead * self.d_k)
        output = self.out_proj(context)
        return output


In [10]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, nhead, dropout)
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src):
        src2 = self.self_attn(src, src, src)
        src = src + self.dropout1(src2)
        src = self.norm1(src)
        src2 = self.linear2(self.dropout(F.relu(self.linear1(src))))
        src = src + self.dropout2(src2)
        src = self.norm2(src)
        return src


In [11]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_layers, dim_feedforward, max_seq_length, dropout=0.1):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, max_seq_length)
        self.layers = nn.ModuleList([EncoderLayer(d_model, nhead, dim_feedforward, dropout) for _ in range(num_layers)])
        self.norm = nn.LayerNorm(d_model)

    def forward(self, src):
        src = self.embedding(src) * math.sqrt(d_model)
        src = self.pos_encoder(src)
        for layer in self.layers:
            src = layer(src)
        return self.norm(src)


In [12]:
import torch.nn.functional as F

class Transformer(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_encoder_layers, dim_feedforward, max_seq_length, num_classes, dropout=0.1):
        super(Transformer, self).__init__()
        self.encoder = Encoder(vocab_size, d_model, nhead, num_encoder_layers, dim_feedforward, max_seq_length, dropout)
        self.classifier = nn.Linear(d_model, num_classes)

    def forward(self, src):
        memory = self.encoder(src)
        output = self.classifier(memory.mean(dim=1))
        return output


In [13]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    input_ids = [torch.tensor(item['input_ids']) for item in batch]
    labels = torch.tensor([item['Rating'] for item in batch])

    input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=0)  

    return {'input_ids': input_ids_padded, 'Rating': labels}

from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, collate_fn=collate_fn)


In [15]:
import torch.optim as optim

vocab_size = 32000
d_model = 512
nhead = 8
num_encoder_layers = 3
dim_feedforward = 2048
max_seq_length = 2056
num_classes = df['Rating'].nunique() 

model = Transformer(vocab_size, d_model, nhead, num_encoder_layers, dim_feedforward, max_seq_length, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 10 

for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device).long() 
        labels = batch['Rating'].to(device)
        outputs = model(input_ids)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch}, Loss: {loss.item()}")

    # Save the model checkpoint after each epoch
    torch.save(model.state_dict(), f'model_epoch_{epoch}.pt')


<ipython-input-13-f85b464598f6>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = [torch.tensor(item['input_ids']) for item in batch]


Streaming output truncated to the last 5000 lines.
Epoch 7, Loss: 0.13564178347587585
Epoch 7, Loss: 0.038309965282678604
Epoch 7, Loss: 0.06130361557006836
Epoch 7, Loss: 0.15654432773590088
Epoch 7, Loss: 0.19872424006462097
Epoch 7, Loss: 0.19811666011810303
Epoch 7, Loss: 0.1279045045375824
Epoch 7, Loss: 0.11796874552965164
Epoch 7, Loss: 0.20172540843486786
Epoch 7, Loss: 0.08036615699529648
Epoch 7, Loss: 0.2013535350561142
Epoch 7, Loss: 0.222664475440979
Epoch 7, Loss: 0.09723581373691559
Epoch 7, Loss: 0.34091421961784363
Epoch 7, Loss: 0.09571424871683121
Epoch 7, Loss: 0.05788399651646614
Epoch 7, Loss: 0.0786089226603508
Epoch 7, Loss: 0.15839940309524536
Epoch 7, Loss: 0.03870310261845589
Epoch 7, Loss: 0.06247948855161667
Epoch 7, Loss: 0.06988964229822159
Epoch 7, Loss: 0.16611023247241974
Epoch 7, Loss: 0.21048793196678162
Epoch 7, Loss: 0.2813991904258728
Epoch 7, Loss: 0.20444238185882568
Epoch 7, Loss: 0.12843084335327148
Epoch 7, Loss: 0.029481759294867516
Epoch 7,

In [31]:
torch.save(model.state_dict(), f'model_epoch_{epoch}.pt')

In [17]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model.eval()
predictions, true_labels = [], []

for batch in test_loader:
    input_ids = batch['input_ids'].long().to(device)
    labels = batch['Rating'].to(device)
    with torch.no_grad():
        outputs = model(input_ids)
    preds = torch.argmax(outputs, dim=1)
    predictions.extend(preds.cpu().numpy())
    true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='weighted')
print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")


<ipython-input-13-f85b464598f6>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = [torch.tensor(item['input_ids']) for item in batch]


Accuracy: 0.848682945587387, Precision: 0.8497160338277708, Recall: 0.848682945587387, F1 Score: 0.8491128108850445


In [32]:
# Load the saved model checkpoint
model = Transformer(vocab_size, d_model, nhead, num_encoder_layers, dim_feedforward, max_seq_length, num_classes)
model.load_state_dict(torch.load('model_epoch_9.pt', map_location=device)) 
model.to(device)  # Move the model to the GPU

model.eval()

predicted_sentiments = []

for batch in test_loader:
    input_ids = batch['input_ids'].to(device)
    with torch.no_grad():
        outputs = model(input_ids)
    logits = outputs.logits if hasattr(outputs, 'logits') else outputs
    predicted_classes = torch.argmax(logits, dim=1).cpu().numpy()
    predicted_sentiments.extend(predicted_classes)

print("Predictions for Test Data:")
for text, sentiment in zip(test_texts, predicted_sentiments):
    print(f"Text: {text}, Predicted Sentiment: {sentiment}")


<ipython-input-13-f85b464598f6>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = [torch.tensor(item['input_ids']) for item in batch]


Streaming output truncated to the last 5000 lines.
Text: كتب لغير المسلمين والباحثين فقط, Predicted Sentiment: 1
Text: ءكثر شي ءحببته ذكره لءسما بناته وحفيدته بداية الكتاب شي لطيف يحمد الكتاب ءهدي دار الشيما بنت الحارث نهاية العام الدراسي, Predicted Sentiment: 1
Text: اعتذار مدير الفندق وحسن تعامله يجعلني افكر بالحجز مرة اخري المدير مغادرتي وتقديمي ملااحظاتي علي التقصير النواحي قدم خدمة التوصيل للمطار بعرض مخفض كاعتذار عدم الرد علي الرسيبشن وعدم تزويدي باغطية لابني, Predicted Sentiment: 1
Text: حب اليغات الجديدة, Predicted Sentiment: 2
Text: قهر انا ساكن الشرايع ويجيب كنتاكي, Predicted Sentiment: 0
Text: جيد الفندق قريب الخدمات الافطار يحتاج للتنويع او يفضل الغا الافطار, Predicted Sentiment: 1
Text: موقع اي مصداقيه طرش طلبيه يقولك موقع خارج نطاق موقعك يوصل مطعم خاصه المطعم طالب مسوي عرض, Predicted Sentiment: 1
Text: توصيل سريع لطيف ومريح, Predicted Sentiment: 2
Text: مرضي التاءخير التشيك ان, Predicted Sentiment: 1
Text: ليه مفيش توصيل لمنطقتنا اسكان اجتماعي مبنية اكتر ٦ سنين ومش بعيدة 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




Text: موقع ءمازون گيدا للكسب, Predicted Sentiment: 0
Text: ءصبحت الخدمه سيءة جدا, Predicted Sentiment: 0
Text: خدمة العملا الممتازة ءسعار كبيرة للشحن السريع تقول الشركة ءيضا, Predicted Sentiment: 2
Text: الممكن ان اعطيها العلامة الكاملة اسلوبها الاكثر سطحي لغتها بسيطة لحد الابتذال تشعر ان كتبها اديبا, Predicted Sentiment: 1
Text: حديث القمر مصطفى صادق الرافعي مصر مليحة لبنان اسمها ليلى ءديبة وشاعرة حبه ءثمر الكتاب يحتوي الكتاب فصول يمازج الكاتب الحب ومواضيع ءخرى تقض فءاده ومع ليلى ءشار الكاتب للاستبداد والسعادة والبءس والفقر والشعر العربي الءلحاد جمال الطبيعة الدموع التجارة بالجمال الانثوي والصداقة لءنه يوجد يستحق الحب الءرض فقد وجه كلامه وحبه للقمر يتسيد كبد السما خواطر نثرية كتبها الرافعي لمساعدة الطلبة فلسفة وءدب فخم وفكر صعب ومعقد تستطيع ءن تفهمها للوهلة الءولى ءو ءنك تحتاج لتركيز شديد وهدو تام تلتقط وتفهم المكتوب ءسلوب رفيع تدل غرور صاحبه تفيد للمهتمين بالءدب والكتابة لتنمية القريحة الءدبية لديهم ءحببت بعضها وسءمني ءغلبها غموض تكتنف الكثير العبارات تستطيع ءن تقرء للرافعي ءلا زاوي

In [35]:
from tokenizers import Tokenizer

# Load the tokenizer
tokenizer = Tokenizer.from_file("/content/tokenizer.json")

def predict_sentiment_with_custom_tokenizer_arabic(input_text_arabic, model, tokenizer, device):
    encoded_input = tokenizer.encode(input_text_arabic)




    input_ids = torch.tensor(encoded_input.ids).unsqueeze(0).to(device) 

    model.eval()
    with torch.no_grad():
        outputs = model(input_ids)

    logits = outputs.logits if hasattr(outputs, 'logits') else outputs
    predicted_class = torch.argmax(logits, dim=1).cpu().item()

    return predicted_class

# Example usage with Arabic input text
input_text_arabic = "كانت هذه المنتجات رائعة وتجربة التسوق كانت سهلة للغاية."
predicted_sentiment_arabic = predict_sentiment_with_custom_tokenizer_arabic(input_text_arabic, model, tokenizer, device)
print(f"Input Text (Arabic): {input_text_arabic}")
print(f"Predicted Sentiment: {predicted_sentiment_arabic}")


Input Text (Arabic): كانت هذه المنتجات رائعة وتجربة التسوق كانت سهلة للغاية.
Predicted Sentiment: 2
